In [45]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [46]:
# Import the datasets
features = pd.read_csv("Features2022.csv")
energy = pd.read_csv("Energy2022.csv")
#energy = pd.read_excel("Energy2022.xlsx", header = 1)
humidity = pd.read_csv("Humidity2022.csv")
irradiance = pd.read_csv("Irradiance2022.csv")

In [48]:
# From the features, select the observations with temperatures and CO2 concentration
temperatures = features[['_S_talo_S111_TE',
       '_S_talo_S112_TE', '_S_talo_S114_TE', '_S_talo_S115_TE',
       '_S_talo_S116_TE', '_S_talo_S117_TE', '_S_talo_S119_TE',
       '_S_talo_S125_TE', '_S_talo_S201_TE', '_S_talo_S202_TE',
       '_S_talo_S210_TE', '_S_talo_S213_TE', '_S_talo_S213b_TE',
       '_S_talo_S214_TE', '_S_talo_S214b_TE', '_S_talo_S215_TE',
       '_S_talo_S301_TE', '_S_talo_S307_TE', '_S_talo_S310_TE',
       '_S_talo_S311_TE', '_S_talo_S311b_TE', '_S_talo_S312_TE',
       '_S_talo_S313_TE', '_S_talo_S316_TE']]

qualities = features[['_S_talo_S111_QE', '_S_talo_S112_QE', '_S_talo_S114_QE',
       '_S_talo_S115_QE', '_S_talo_S116_QE', '_S_talo_S117_QE',
       '_S_talo_S119_QE', '_S_talo_S125_QE', '_S_talo_S201_QE',
       '_S_talo_S202_QE', '_S_talo_S210_QE', '_S_talo_S213_QE',
       '_S_talo_S213b_QE', '_S_talo_S214_QE', '_S_talo_S214b_QE',
       '_S_talo_S215_QE', '_S_talo_S301_QE', '_S_talo_S307_QE',
       '_S_talo_S310_QE', '_S_talo_S311_QE', '_S_talo_S311b_QE',
       '_S_talo_S312_QE', '_S_talo_S313_QE', '_S_talo_S316_QE']]

# Multiply some room observations of CO2 concentration to normalize to them scale
for i in ["_S_talo_S111_QE", "_S_talo_S114_QE", "_S_talo_S115_QE",
          "_S_talo_S116_QE", "_S_talo_S117_QE", "_S_talo_S119_QE", 
          "_S_talo_S213b_QE", "_S_talo_S214b_QE"]:
    qualities.loc[:, i] = qualities.loc[:, i] * 1000
    
# Calculate the mean of the room observations  
temperatures.loc[:, "Inside_temperature"] = temperatures.mean(1)   
qualities.loc[:, "CO2_concentration"] = qualities.mean(1)

# Select and rename the remaining sensors
features = features[['Time', '_S_talo_S_LV02_TE03', '_S_talo_S_LV01_TE03',
       '_S_talo_S_LIV01_TE03', '_S_talo_U_TE90', '_S_talo_S_KLV01_TE01',
       '_S_talo_S_LKV01_TV01', '_S_talo_S_LKV01_TE03']]

features.rename(columns={'_S_talo_S_LV02_TE03': 'Floor_network_2_temperature', 
                         '_S_talo_S_LV01_TE03': 'Radiator_network_1_temperature',
                         "_S_talo_S_LIV01_TE03": "Ventilation_network_1_temperature",
                         "_S_talo_U_TE90": "Outside_temperature_1",
                         "_S_talo_S_KLV01_TE01": "District_heat_temperature",
                         "_S_talo_S_LKV01_TV01": "Domestic_water_network_1_primary_valve",
                         "_S_talo_S_LKV01_TE03": "Domestic_water_network_1_temperature"
                        }, inplace=True)

# Merge the temperature, CO2 concentration and other features into the same dataset
features = pd.concat([features, temperatures["Inside_temperature"], qualities["CO2_concentration"]], axis = 1)

# Set the correct timezone (GMT) for the feature data
features["Time"] = pd.to_datetime(features.Time, format = '%Y-%m-%d %H:%M:%S')
features["Time"] = features["Time"].dt.tz_localize('UTC', ambiguous='infer')

# Set the correct timezone (Finnish) for the energy data
#energy["Aika"] = energy["Aika"].str.replace("\+3", '', regex = True)
#energy["Aika"] = energy["Aika"].str[3:]
#energy["Aika"] = energy["Aika"].str[3:-1]
energy["Aika"] = pd.to_datetime(energy.Aika, format = '%Y-%m-%d %H:%M:%S')
#energy["Aika"] = pd.to_datetime(energy.Aika, format = '%d.%m.%Y %H.%M.%S')
energy["Time"] = energy["Aika"].dt.tz_localize('Europe/Helsinki', ambiguous='infer').dt.tz_convert('UTC')

# Merge the features and the energy into single dataset
result = pd.merge(features, energy[["kWh", "Time"]], how = "left", on = "Time")

# Rename the energy column
result.rename(columns = {"kWh" : "Energy_consumption"}, inplace = True)

# Merge the humidity and irradiance datasets with the general dataset
def data_merge(main_dataset, dataset, parameter):
    dataset["Aika"] = dataset["Year"].astype(str) + "-" + dataset["m"].astype(str) + "-" + dataset["d"].astype(str) + " " + dataset["Time"]
    dataset["Aika"] = pd.to_datetime(dataset.Aika, format = '%Y.%m.%d %H:%M')
    dataset["Time"] = dataset["Aika"].dt.tz_localize('UTC', ambiguous='infer')
    main_dataset = pd.merge(main_dataset, dataset[[parameter, "Time"]], how = "left", on = "Time")
    return main_dataset

result = data_merge(result, humidity, "Relative humidity (%)")
result = data_merge(result, irradiance, "Direct solar radiation (W/m2)")

# Rename the remaining columns
result.rename(columns = {"Direct solar radiation (W/m2)" : "Solar_irradiance", 
                           "Relative humidity (%)" : "Outside_humidity", 
                           "Outside_temperature_1" : "Outside_temperature_average"}, inplace = True)

# Detect and remove the missing window when all sensors show zero
datana = result[(result["Outside_temperature_average"] == 0) & (result["Floor_network_2_temperature"] == 0)]
result = result.drop(index = datana.index)
result.reset_index(inplace = True)

In [55]:
result

,Time,Floor_network_2_temperature,Radiator_network_1_temperature,Ventilation_network_1_temperature,Outside_temperature_average,District_heat_temperature,Domestic_water_network_1_primary_valve,Domestic_water_network_1_temperature,Inside_temperature,CO2_concentration,Energy_consumption,Outside_humidity,Solar_irradiance
0,2021-11-29 09:00:00+00:00,31.4,50.2,46.2,-7.300,91.0,27.3,58.0,21.829167,490.666667,NaN,88,11.2
1,2021-11-29 10:00:00+00:00,30.4,50.8,45.5,-6.700,90.1,28.3,57.9,21.829167,492.291667,112.0,88,7.3
2,2021-11-29 11:00:00+00:00,31.0,50.7,46.7,-7.000,90.4,28.1,58.0,21.891667,514.583333,102.0,89,164.4
3,2021-11-29 12:00:00+00:00,30.5,50.3,45.6,-7.200,90.3,27.0,57.9,21.950000,542.583333,104.0,88,1.7
4,2021-11-29 13:00:00+00:00,31.1,51.2,46.9,-7.400,90.5,27.5,58.1,21.883333,499.916667,101.0,87,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,2022-02-09 09:00:00+00:00,29.0,40.2,40.0,-1.100,87.9,27.4,58.0,21.745833,521.991667,NaN,90,410.3
1729,2022-02-09 10:00:00+00:00,28.4,39.5,40.1,-0.220,88.4,30.2,57.7,21.837500,514.583333,NaN,85,690.6
1730,2022-02-09 11:00:00+00:00,28.2,38.4,38.8,0.744,87.7,29.0,57.8,21.858333,538.375000,NaN,84,6.4
1731,2022-02-09 12:00:00+00:00,28.0,36.3,37.1,2.240,86.1,28.1,57.9,21.820833,532.583333,NaN,88,1.0


In [50]:
result = result.drop(columns = ["index"])

In [51]:
result.isna().any()

Time                                      False
Floor_network_2_temperature               False
Radiator_network_1_temperature            False
Ventilation_network_1_temperature         False
Outside_temperature_average               False
District_heat_temperature                 False
Domestic_water_network_1_primary_valve    False
Domestic_water_network_1_temperature      False
Inside_temperature                        False
CO2_concentration                         False
Energy_consumption                         True
Outside_humidity                          False
Solar_irradiance                          False
dtype: bool

In [53]:
result[result.isna().any(axis=1)].index

Int64Index([   0, 1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728,
            1729, 1730, 1731, 1732],
           dtype='int64')

In [54]:
result.drop(index = result[result.isna().any(axis=1)].index)

,Time,Floor_network_2_temperature,Radiator_network_1_temperature,Ventilation_network_1_temperature,Outside_temperature_average,District_heat_temperature,Domestic_water_network_1_primary_valve,Domestic_water_network_1_temperature,Inside_temperature,CO2_concentration,Energy_consumption,Outside_humidity,Solar_irradiance
1,2021-11-29 10:00:00+00:00,30.4,50.8,45.5,-6.700,90.1,28.3,57.9,21.829167,492.291667,112.0,88,7.3
2,2021-11-29 11:00:00+00:00,31.0,50.7,46.7,-7.000,90.4,28.1,58.0,21.891667,514.583333,102.0,89,164.4
3,2021-11-29 12:00:00+00:00,30.5,50.3,45.6,-7.200,90.3,27.0,57.9,21.950000,542.583333,104.0,88,1.7
4,2021-11-29 13:00:00+00:00,31.1,51.2,46.9,-7.400,90.5,27.5,58.1,21.883333,499.916667,101.0,87,0.6
5,2021-11-29 14:00:00+00:00,31.0,50.1,46.4,-7.500,91.0,28.9,57.9,21.754167,462.375000,96.0,87,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,2022-02-08 19:00:00+00:00,28.2,38.8,38.6,0.655,83.5,28.3,58.0,21.387500,437.916667,64.0,96,0.7
1715,2022-02-08 20:00:00+00:00,28.2,38.3,38.7,0.749,83.0,29.7,58.0,21.379167,446.666667,62.0,96,0.3
1716,2022-02-08 21:00:00+00:00,28.2,37.9,38.6,0.712,82.7,28.1,58.0,21.337500,444.666667,63.0,97,0.4
1717,2022-02-08 22:00:00+00:00,28.1,38.3,38.5,0.449,82.6,28.3,58.0,21.337500,442.416667,59.0,90,-0.8


In [56]:
result = result.drop(index = result[result.isna().any(axis=1)].index)

In [57]:
result

,Time,Floor_network_2_temperature,Radiator_network_1_temperature,Ventilation_network_1_temperature,Outside_temperature_average,District_heat_temperature,Domestic_water_network_1_primary_valve,Domestic_water_network_1_temperature,Inside_temperature,CO2_concentration,Energy_consumption,Outside_humidity,Solar_irradiance
1,2021-11-29 10:00:00+00:00,30.4,50.8,45.5,-6.700,90.1,28.3,57.9,21.829167,492.291667,112.0,88,7.3
2,2021-11-29 11:00:00+00:00,31.0,50.7,46.7,-7.000,90.4,28.1,58.0,21.891667,514.583333,102.0,89,164.4
3,2021-11-29 12:00:00+00:00,30.5,50.3,45.6,-7.200,90.3,27.0,57.9,21.950000,542.583333,104.0,88,1.7
4,2021-11-29 13:00:00+00:00,31.1,51.2,46.9,-7.400,90.5,27.5,58.1,21.883333,499.916667,101.0,87,0.6
5,2021-11-29 14:00:00+00:00,31.0,50.1,46.4,-7.500,91.0,28.9,57.9,21.754167,462.375000,96.0,87,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,2022-02-08 19:00:00+00:00,28.2,38.8,38.6,0.655,83.5,28.3,58.0,21.387500,437.916667,64.0,96,0.7
1715,2022-02-08 20:00:00+00:00,28.2,38.3,38.7,0.749,83.0,29.7,58.0,21.379167,446.666667,62.0,96,0.3
1716,2022-02-08 21:00:00+00:00,28.2,37.9,38.6,0.712,82.7,28.1,58.0,21.337500,444.666667,63.0,97,0.4
1717,2022-02-08 22:00:00+00:00,28.1,38.3,38.5,0.449,82.6,28.3,58.0,21.337500,442.416667,59.0,90,-0.8


In [58]:
#result.drop(result.tail(1000).index,inplace=True)

#### We discard:
- 339 observations from 2020.02.06 08:00 to 2020.02.20 14:00 for all the building sensors showing zero. <br>
- Total of 202 observations by FMI of solar irradiance and humidity together have missing data. <br>

In [59]:
result.to_csv("s-building_data.csv", index = False)